## Importing the Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import json
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, auc, classification_report
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

## Importing variables from the preprocessing notebook as csv files

In [ ]:
X_train = np.loadtxt('X_train.csv', delimiter=',')
X_test = np.loadtxt('X_test.csv', delimiter=',')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')